In [ ]:
!pip install datasets -q
from torch.utils.data import DataLoader, Dataset, random_split
import random
import numpy as np
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import numpy as np
from collections import defaultdict
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from datasets import load_dataset
from torchvision.models import resnet18, resnet50,resnet152, densenet121, densenet201
import os
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.optim import Adam
import torch
from tqdm import tqdm
from sklearn.metrics import f1_score
import pandas as pd

# Set seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:

# Function to check if an image is blurry
def is_blurry(image, threshold=90.0):
    # Ensure the image is a NumPy array
    if not isinstance(image, np.ndarray):
        image = np.array(image)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray_image, cv2.CV_64F).var()
    return laplacian_var < threshold


# Function to compute image hash
def dhash(image, hash_size=8):
    # Ensure the image is a NumPy array
    if not isinstance(image, np.ndarray):
        image = np.array(image)
    # Convert to grayscale if necessary
    if len(image.shape) == 3:  # Check if the image has color channels
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(image, (hash_size + 1, hash_size))
    diff = resized[:, 1:] > resized[:, :-1]
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])


# Load dataset
dataset = load_dataset('hmdliu/ACAC-4K')

# Filter out blurry images
non_blurry_images = [example for example in dataset['train'] if not is_blurry(example['image'])]


# Remove duplicates
hashes = defaultdict(list)
for idx, example in enumerate(non_blurry_images):
    image_hash = dhash(example['image'])
    hashes[image_hash].append(example)

# Remove duplicates by keeping only one example per hash
unique_images = [examples[0] for examples in hashes.values()]


# Custom Dataset Class
class CustomHFDataset(Dataset):
    def __init__(self, hf_dataset, transform=None, test_flag=False):
        self.hf_dataset = hf_dataset
        self.transform = transform
        self.test_flag = test_flag

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        example = self.hf_dataset[idx]
        image = example['image']
        label = example['label']
        if self.transform:
            image = self.transform(image)
        if self.test_flag:
            return image, example['idx']
        else:
            return image, label

# Define transformations with data augmentation
transform_train = transforms.Compose([
    transforms.Resize(512),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.6127, 0.5398, 0.4442], std=[0.1288, 0.1147, 0.1009])
])

transform_test = transforms.Compose([
    transforms.Resize(512),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.6127, 0.5398, 0.4442], std=[0.1288, 0.1147, 0.1009])
])

train_data = CustomHFDataset(unique_images, transform=transform_train)
test_data = CustomHFDataset(dataset['test'], transform=transform_test)

# Split the training data
num_samples = len(train_data)
test_split_size = 200
train_split_size = num_samples - test_split_size

train_dataset, extra_test_dataset = random_split(
    train_data,
    [train_split_size, test_split_size],
    generator=torch.Generator().manual_seed(SEED)
)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(extra_test_dataset, batch_size=16, shuffle=False)
kaggle_test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

print(f"Train set size: {len(train_dataset)}")
print(f"Extra test set size: {len(extra_test_dataset)}")

Train set size: 2885
Extra test set size: 200


In [ ]:
# from torchvision.models import vit_b_16

# Initialize models without pretrained weights
ResNet18_cleaner = resnet18(weights=None)
models = {
    "ResNet18": resnet18(weights=None),
    "ResNet50": resnet50(weights=None),
    "ResNet152": resnet152(weights=None),
    "DenseNet121": densenet121(weights=None),
    "DenseNet201": densenet201(weights=None),
    # "ViT": vit_b_16(weights=None)
}

# Modify the final layer to match the number of classes (e.g., 10 classes)
num_classes = len(dataset['train'].features['label'].names)


# Iterate through the models and adjust the final layer
for name, model in models.items():
    if 'ResNet' in name or 'DenseNet' in name:
        if hasattr(model, 'fc'):  # ResNet
            model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
        elif hasattr(model, 'classifier'):  # DenseNet
            model.classifier = torch.nn.Linear(model.classifier.in_features, num_classes)
    elif 'ViT' in name:  # For ViT (Vision Transformer)
        # Check for the attribute to modify
        if hasattr(model, 'head'):  # ViT model with head
            model.head = torch.nn.Linear(model.head.in_features, num_classes)
        elif hasattr(model, 'classifier'):  # ViT model with classifier
            model.classifier = torch.nn.Linear(model.classifier.in_features, num_classes)

    # Print the updated model architecture
    print(f"Updated {name} model:")

models = {name: model.to(device) for name, model in models.items()}


Updated ResNet18 model:
Updated ResNet50 model:
Updated ResNet152 model:
Updated DenseNet121 model:
Updated DenseNet201 model:


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

drive_model_dir = "/kaggle/models"

In [ ]:

class FocalLoss(nn.Module):
    def __init__(self, gamma=3, alpha=0.2):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, inputs, targets):
        BCE_loss = F.cross_entropy(inputs, targets, reduction='mean')
        pt = torch.exp(-BCE_loss)
        focal_loss = ((1 - pt) ** self.gamma * BCE_loss).mean()


        return focal_loss.mean()



criterion = FocalLoss()




In [ ]:
def train_model(model, optimizer, criterion, train_loader):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)

def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    return all_preds, all_labels

def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()


In [ ]:
# Evaluate Function
def test_evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return print(name,":",100 * correct / total)

In [ ]:
def train_and_save_model(model, model_name, train_loader, num_epochs=12):
    optimizer = Adam(model.parameters(), lr=0.0005)
    criterion = FocalLoss()

    # criterion = nn.CrossEntropyLoss()
    # optimizer = torch.optim.Adam(model.parameters(), lr=0.00005)
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}")

    # Save the model state to Google Drive
    test_evaluate_model(model, test_loader)
    model_path = f'{drive_model_dir}/{model_name}.pth'
    torch.save(model.state_dict(), model_path)
    print(f"{model_name} saved to {model_path}")

# Train and save each model one by one
# for name in models:
name = "ResNet18_cleaner"
print(f"Training {name}...")
model = ResNet18_cleaner.to(device)
train_and_save_model(model, name, train_loader,4)
del model  # Delete the model to free up GPU memory
torch.cuda.empty_cache()  # Clear GPU cache




ResNet18_cleaner saved to /kaggle/models/ResNet18_cleaner.pth


In [ ]:
accuracy_extra_test = test_evaluate_model(ResNet18_cleaner, test_loader)

ResNet18_cleaner : 58.5


In [ ]:
import torch.nn.functional as F
from torch.utils.data import Subset

def test_and_filter(model, test_loader, train_dataset, threshold=0.4):
    """
    Test the model and identify high-confidence incorrect predictions.
    Remove these samples from the training dataset.
    """
    model.eval()
    incorrect_high_confidence_indices = []

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = F.softmax(outputs, dim=1)  # Get probabilities
            preds = outputs.argmax(dim=1)  # Get predicted class indices

            # Check for high-confidence incorrect predictions
            for i in range(len(labels)):
                if preds[i] != labels[i] and probs[i][preds[i]] > threshold:
                    # Calculate global index in test dataset
                    global_idx = batch_idx * test_loader.batch_size + i
                    incorrect_high_confidence_indices.append(global_idx)

    print(f"Found {len(incorrect_high_confidence_indices)} high-confidence incorrect predictions.")

    # Filter out these indices from the training dataset
    train_dataset = filter_training_data(train_dataset, incorrect_high_confidence_indices)

    return train_dataset


def filter_training_data(train_dataset, indices_to_remove):
    """
    Remove specified indices from the training dataset.
    """
    # Create a new dataset excluding the specified indices
    all_indices = set(range(len(train_dataset)))
    filtered_indices = list(all_indices - set(indices_to_remove))

    filtered_dataset = Subset(train_dataset, filtered_indices)
    print(f"Training set reduced from {len(train_dataset)} to {len(filtered_dataset)} samples.")

    return filtered_dataset


# Example usage
# for name in models:
name = "ResNet18_cleaner"
print(f"Testing and filtering for {name}...")

    # Load model and test it
model = ResNet18_cleaner.to(device)
train_dataset = test_and_filter(model, test_loader, train_dataset)
new_train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)



Testing and filtering for ResNet18_cleaner...
Found 27 high-confidence incorrect predictions.
Training set reduced from 2844 to 2817 samples.


In [ ]:
len(train_dataset)

2817

In [ ]:
from sklearn.metrics import f1_score
import numpy as np

def train_and_save_model(model, model_name, train_loader, test_loader, num_epochs=12):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00005)
    criterion = FocalLoss(alpha=1.0, gamma=2.0, reduction='mean')  # Use Focal Loss


    # Training loop
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}")

    # Evaluate the model on the test set to calculate F1 scores
    f1_scores = evaluate_f1_scores(model, test_loader)

    # Normalize F1 scores to get weights
    weights = f1_scores / np.sum(f1_scores)
    print(f"Class-wise F1 scores for {model_name}: {f1_scores}")
    print(f"Normalized weights for {model_name}: {weights}")
    test_evaluate_model(model, test_loader)

    # Save the model state to Google Drive
    model_path = f'{drive_model_dir}/{model_name}.pth'
    torch.save(model.state_dict(), model_path)
    print(f"{model_name} saved to {model_path}")

    return weights


def evaluate_f1_scores(model, test_loader):
    """Evaluate class-wise F1 scores on the test set."""
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    # Compute F1 scores for each class
    f1_scores = f1_score(all_labels, all_preds, average=None)  # Class-wise F1 scores
    return np.array(f1_scores)


# Train and save each model one by one while calculating weights
model_weights = {}
for name in models:
    print(f"Training {name}...")
    model = models[name].to(device)

    # Train and compute weights based on class-wise F1 scores
    weights = train_and_save_model(model, name, new_train_loader, test_loader)

    # Store weights for the ensemble voting system
    model_weights[name] = weights

    del model  # Delete the model to free up GPU memory
    torch.cuda.empty_cache()  # Clear GPU cache

print("Model training complete. Weights for ensemble:")
print(model_weights)

# The `model_weights` dictionary now contains normalized class-wise weights for each model.


In [ ]:
print("Model training complete. Weights for ensemble:")
print(model_weights)

Model training complete. Weights for ensemble:
{'ResNet18': array([0.        , 0.2640688 , 0.09841073, 0.12574705, 0.24231547,
       0.26945795]), 'ResNet50': array([0.        , 0.28690981, 0.        , 0.16074228, 0.27783201,
       0.2745159 ])}


In [ ]:
def test(model, test_loader, device):
    model.eval()
    predictions = []
    idxs = []

    with torch.no_grad():
        for images, idx in tqdm(test_loader, desc="Testing"):
            images = images.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)

            predictions.extend(predicted.cpu().numpy())
            idxs.extend(idx.numpy())  # Ensure idx is on CPU

    return predictions, idxs

In [ ]:
model_predictions = {}

# Generate predictions for each model
for name, model in models.items():
    print(f"Generating predictions with {name}...")
    preds, _ = test(model, test_loader, device)  # Use test function to get predictions
    model_predictions[name] = preds  # Store predictions


In [ ]:
def weighted_voting(model_predictions, weights):
    num_classes = len(next(iter(weights.values())))  # Number of classes (assumes consistent across models)
    final_predictions = []

    # Iterate through each sample
    for i in range(len(next(iter(model_predictions.values())))):
        class_votes = np.zeros(num_classes)  # Initialize votes for all classes

        # Accumulate weighted votes from each model
        for model_name in model_predictions:
            pred_class = model_predictions[model_name][i]
            class_votes[pred_class] += weights[model_name][pred_class]

        # Select the class with the maximum weighted vote
        final_predictions.append(np.argmax(class_votes))

    return final_predictions


In [ ]:
# Perform weighted voting
ensemble_predictions = weighted_voting(model_predictions, model_weights)

In [ ]:
import pandas as pd

# Save predictions to CSV
# df = pd.DataFrame({'idx': range(len(ensemble_predictions)), 'predicted_label': ensemble_predictions})
# csv_path = '/ensemble_local_test.csv'
# df.to_csv(csv_path, index=False)
# print(f"Predictions saved to {csv_path}")

# Extract ground truth labels from the extra_test_dataset
# Extract ground truth labels from extra_test_dataset
true_labels = [label for _, label in extra_test_dataset]  # Access the second element (label) in each tuple
  # Corrected list comprehension

# Evaluate ensemble predictions
from sklearn.metrics import classification_report
print(classification_report(true_labels, ensemble_predictions))


In [ ]:
model_predictions = {}
all_indices = None  # To store indices from one of the models (should be consistent across models)

for name, model in models.items():
    print(f"Generating predictions with {name}...")
    preds, idxs = test(model, kaggle_test_loader, device)  # Get predictions and indices
    model_predictions[name] = preds  # Store predictions

    # Save indices from the first model (all models should have consistent indices)
    if all_indices is None:
        all_indices = idxs

ensemble_predictions = weighted_voting(model_predictions, model_weights)

import pandas as pd

# Save predictions to CSV
df = pd.DataFrame({'idx': all_indices, 'predicted_label': ensemble_predictions})
csv_path = 'ensemble_kaggle_test.csv'
df.to_csv(csv_path, index=False)
print(f"Predictions saved to {csv_path}")

Hyperparameter Tunning


In [ ]:
import torch
import random
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from datasets import load_dataset
from tqdm import tqdm
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F


# Set a fixed random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# Define the Custom Dataset Class
class CustomHFDataset(Dataset):
    def __init__(self, hf_dataset, transform=None, test_flag=False):
        """
        Args:
            hf_dataset: A split of the Hugging Face dataset (e.g., train, val, test)
            transform: PyTorch transforms to apply on the images
            test_flag: If True, returns indices instead of labels (for testing).
        """
        self.hf_dataset = hf_dataset
        self.transform = transform
        self.test_flag = test_flag

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        example = self.hf_dataset[idx]
        image = example['image']
        label = example.get('label', -1)  # Default to -1 if no label exists (e.g., test set)
        index = example.get('idx', idx)  # Default to index if 'idx' is not explicitly provided

        if self.transform:
            image = self.transform(image)

        if self.test_flag:
            return image, index  # Return image and index for testing
        else:
            return image, label  # Return image and label for training


# Define Focal Loss Class
class FocalLoss(nn.Module):
    def __init__(self, alpha=1.0, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        logpt = F.log_softmax(inputs, dim=1)
        pt = torch.exp(logpt)
        logpt = logpt.gather(1, targets.unsqueeze(1)).squeeze(1)
        pt = pt.gather(1, targets.unsqueeze(1)).squeeze(1)

        focal_loss = -self.alpha * (1 - pt) ** self.gamma * logpt

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


# Define Model Trainer Class
class ModelTrainer:
    def __init__(self, model, criterion, optimizer, device):
        """
        Args:
            model: PyTorch model to train.
            criterion: Loss function.
            optimizer: Optimizer for training.
            device: Device to run training on (CPU/GPU).
        """
        self.model = model.to(device)
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = device

    def train(self, train_loader, num_epochs=12):
        """
        Train the model for a specified number of epochs.

        Args:
            train_loader: DataLoader for training data.
            num_epochs: Number of epochs to train.

        Returns:
            Trained model.
        """
        for epoch in range(num_epochs):
            self.model.train()
            running_loss = 0.0

            for images, labels in tqdm(train_loader):
                images, labels = images.to(self.device), labels.to(self.device)

                # Zero gradients
                self.optimizer.zero_grad()

                # Forward pass and compute loss
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)

                # Backward pass and optimization step
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()

            print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")

    def evaluate(self, test_loader):
        """
        Evaluate the model on a test set.

        Args:
            test_loader: DataLoader for test data.

        Returns:
            predictions: List of predicted labels.
            true_labels: List of ground truth labels.
        """
        self.model.eval()

        predictions = []
        true_labels = []

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(self.device), labels.to(self.device)

                outputs = self.model(images)
                _, predicted = outputs.max(1)

                predictions.extend(predicted.cpu().numpy())
                true_labels.extend(labels.cpu().numpy())

        return predictions, true_labels


# Define Utility Functions
def save_predictions_to_csv(predictions, dataset_test_split, output_path="test_predictions.csv"):
    """
    Save predictions along with indices to a CSV file.

    Args:
      predictions: List of predicted labels.
      dataset_test_split: Test dataset containing indices.
      output_path: Path to save the CSV file.
    """
    idx_column = [data['idx'] for data in dataset_test_split]

    results_df = pd.DataFrame({
      'idx': idx_column,
      'predicted_label': predictions,
    })

    results_df.to_csv(output_path, index=False)
    print(f"Predictions saved to {output_path}!")


In [ ]:

# Load Dataset and Define Transforms
dataset = load_dataset('hmdliu/ACAC-4K')

train_transform = transforms.Compose([
transforms.ColorJitter(brightness=0.2, contrast=0.2),
transforms.ToTensor(),
transforms.Normalize(mean=[0.6127, 0.5398, 0.4442], std=[0.1288, 0.1147, 0.1009]),    ])

test_transform = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize(mean=[0.6127, 0.5398, 0.4442], std=[0.1288, 0.1147, 0.1009]),])


train_dataset = CustomHFDataset(dataset['train'], transform=train_transform)
test_dataset_split = dataset['test']
test_dataset = CustomHFDataset(test_dataset_split, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)


num_classes = len(set(dataset['train']['label']))

model_resnet18 = models.resnet18(weights=None)
model_resnet18.fc = nn.Linear(model_resnet18.fc.in_features, num_classes)

device_used = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion_focal_loss = FocalLoss(alpha=1.0)

optimizer_adam_resnet18 = torch.optim.Adam(model_resnet18.parameters(), lr=5e-5)

trainer_resnet18 = ModelTrainer(model=model_resnet18,
                                    criterion=criterion_focal_loss,
                                    optimizer=optimizer_adam_resnet18,
                                    device=device_used)
trainer_resnet18.train(train_loader=train_loader)
preds_resnet18, true_labels_resnet18 = trainer_resnet18.evaluate(test_loader=test_loader)
save_predictions_to_csv(predictions=preds_resnet18,
                            dataset_test_split=test_dataset_split,
                            output_path="test_predictions.csv")

model_name = "resnet_hyper"
model_path = f'{drive_model_dir}/{model_name}.pth'
torch.save(model.state_dict(), model_path)
print(f"{model_name} saved to {model_path}")
